# Creating Morgan FP 
This notebook runs through how to create the morgan features from rdkit smiles strings (binding) 
Everything should be repeated with the inhibition data. 
- import seaborn 
- Using the data provided by ATOM
- The data directory contains:
  - dfa

In [1]:
from imports import *

In [2]:
bind_data_path = '../data/binding/'
inhib_data_path='../data/inhibition/'
binding_moe = pd.read_csv(bind_data_path+'NEK2_1_uM_min_50_pct_binding_with_moe_descriptors.csv') 
inhibition_moe = pd.read_csv(inhib_data_path+'NEK2_1_uM_min_50_pct_inhibition_with_moe_descriptors.csv')

all_fold_bind = make_splits(binding_moe, bind_data_path, 'binding_random_splits.csv')
all_fold_inhib = make_splits(inhibition_moe, inhib_data_path, 'inhibition_random_splits.csv')

dataset shape:  (1408, 309)
active
0    1351
1      57
Name: count, dtype: int64
Number of minority samples: 57
Majority shape: (1351, 309)
Minority shape: (57, 309)
Majority fold counts: fold
fold1    271
fold4    270
fold2    270
fold3    270
fold5    270
Name: count, dtype: int64
Minority fold counts: fold
fold1    12
fold2    12
fold5    11
fold4    11
fold3    11
Name: count, dtype: int64
after major+minor concat:
all_fold shape: (1408, 310)
all_fold active value counts: active
0    1351
1      57
Name: count, dtype: int64
dataset shape:  (2044, 309)
active
0    1904
1     140
Name: count, dtype: int64
Number of minority samples: 140
Majority shape: (1904, 309)
Minority shape: (140, 309)
Majority fold counts: fold
fold4    381
fold1    381
fold2    381
fold3    381
fold5    380
Name: count, dtype: int64
Minority fold counts: fold
fold4    28
fold3    28
fold1    28
fold5    28
fold2    28
Name: count, dtype: int64
after major+minor concat:
all_fold shape: (2044, 310)
all_fold acti

/Users/jayceepang/msse/chem283/nek2_final/nek2_final/notebooks/../../split_data.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_major.loc[df_major.index[v_ind], 'fold'] = f"fold{i+1}"
/Users/jayceepang/msse/chem283/nek2_final/nek2_final/notebooks/../../split_data.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_minor.loc[df_minor.index[v_ind], 'fold'] = f"fold{i+1}"
/Users/jayceepang/msse/chem283/nek2_final/nek2_final/notebooks/../../split_data.py:28: SettingWithCopyWarning: 
A value is tryin

# Normalize the data create Morgan fingerprints with radius 2 
- Save the normalized data (performs pka adjustments and normalizes to tautomers)
- Save Morgan fingerprints 

In [3]:
smile_prop_df = pd.read_csv('../data/binding/binding_random_splits.csv')
# smile prop train will be normalized feats
smile_prop_train = smile_prop_df[smile_prop_df['fold'] != 'fold1'].drop(columns=['fold'])
smile_prop_test = smile_prop_df[smile_prop_df['fold'] == 'fold1'].drop(columns=['fold'])
mol_train = []
for i in smile_prop_train.index:
    m = Chem.MolFromSmiles(smile_prop_train.loc[i, 'base_rdkit_smiles'])
    if m:
        m.SetProp("_Name", smile_prop_train.loc[i, 'compound_id'])
        m.SetProp("Active", str(smile_prop_train.loc[i, 'active']))
        mol_train.append(utils.normalize(m))

mol_test = []
for i in smile_prop_test.index:
    m = Chem.MolFromSmiles(smile_prop_test.loc[i, 'base_rdkit_smiles'])
    if m:
        m.SetProp("_Name", smile_prop_test.loc[i, 'compound_id'])
        m.SetProp("Active", str(smile_prop_test.loc[i, 'active']))
        mol_test.append(utils.normalize(m))
# normalized feats from smile_prop_train
X_existing_train = smile_prop_train.drop(columns=['compound_id', 'base_rdkit_smiles', 'active'])
X_existing_test = smile_prop_test.drop(columns=['compound_id', 'base_rdkit_smiles', 'active'])
y_train = smile_prop_train['active']
y_test = smile_prop_test['active']


# generate fingeprints: Morgan fingerprint with radius 2
fps = [Chem.AllChem.GetMorganFingerprintAsBitVect(m, 2) for m in mol_train]
# convert the RDKit explicit vectors into numpy arrays
X_morgan_train = []
for fp in fps:
  arr = np.zeros((1,))
  Chem.DataStructs.ConvertToNumpyArray(fp, arr)
  X_morgan_train.append(arr)

# generate fingeprints: Morgan fingerprint with radius 2
fps = [Chem.AllChem.GetMorganFingerprintAsBitVect(m, 2) for m in mol_test]
# convert the RDKit explicit vectors into numpy arrays
X_morgan_test = []
for fp in fps:
  arr = np.zeros((1,))
  Chem.DataStructs.ConvertToNumpyArray(fp, arr)
  X_morgan_test.append(arr)

[16:57:50] Initializing MetalDisconnector
[16:57:50] Running MetalDisconnector
[16:57:50] Initializing Normalizer
[16:57:50] Running Normalizer
[16:57:50] Running LargestFragmentChooser
[16:57:50] Running Uncharger
[16:57:50] Initializing MetalDisconnector
[16:57:50] Running MetalDisconnector
[16:57:50] Initializing Normalizer
[16:57:50] Running Normalizer
[16:57:50] Initializing MetalDisconnector
[16:57:50] Running MetalDisconnector
[16:57:50] Initializing Normalizer
[16:57:50] Running Normalizer
[16:57:50] Running LargestFragmentChooser
[16:57:50] Running Uncharger
[16:57:50] Initializing MetalDisconnector
[16:57:50] Running MetalDisconnector
[16:57:50] Initializing Normalizer
[16:57:50] Running Normalizer
[16:57:50] Initializing MetalDisconnector
[16:57:50] Running MetalDisconnector
[16:57:50] Initializing Normalizer
[16:57:50] Running Normalizer
[16:57:50] Running LargestFragmentChooser
[16:57:50] Running Uncharger
[16:57:50] Initializing MetalDisconnector
[16:57:50] Running MetalD

In [30]:
trainX_norm = X_existing_train.to_numpy()
# y_train = y_train.to_numpy().reshape(-1)
testX_norm = X_existing_test.to_numpy()
# y_test = y_test.to_numpy().reshape(-1)
# we also scale the data

# back to df for saving 

train_x_df = pd.DataFrame(trainX_norm) 
train_y_df = pd.DataFrame(y_train) 
test_y_df = pd.DataFrame(y_test)
test_x_df = pd.DataFrame(testX_norm)

# scale the data for consistency (this is what i did in the project)  using StandardScalar 
train_x_temp, train_y_temp, test_x_temp, test_y_temp = scale_data(train_x_df, train_y_df, test_x_df, test_y_df)

train_x_df = pd.DataFrame(train_x_temp) 
train_y_df = pd.DataFrame(train_y_temp) 
test_y_df = pd.DataFrame(test_y_temp)
test_x_df = pd.DataFrame(test_x_temp)

train_x_df.to_csv(bind_data_path+'splits/bind_train_x_normalized.csv', index=False, header=False)
train_y_df.to_csv(bind_data_path+'splits/bind_train_y_normalized.csv', index=False, header=False) 
test_y_df.to_csv(bind_data_path+'splits/bind_test_y_normalized.csv',index=False, header=False) 
test_x_df.to_csv(bind_data_path+'splits/bind_test_x_normalized.csv', index=False, header=False) 
print(train_x_df.shape, train_y_df.shape, test_x_df.shape, test_x_df.shape)



(1125, 306) (1125, 1) (283, 306) (283, 306)


# Now we will use SMOTE, RandomUndersampler, and ADASYN to treat the datasets (norm MOE and the Morgan FP features)
- Note that ideally, we would have followed this process. In reality, we used the 

In [31]:
train_x = train_x_df.to_numpy()
train_y = train_y_df.to_numpy().reshape(-1)
test_x = test_x_df.to_numpy()
test_y = test_y_df.to_numpy().reshape(-1)
print(train_x.shape, train_y.shape, test_x.shape, test_y.shape)

(1125, 306) (1125,) (283, 306) (283,)


# SMOTE 

In [32]:

split_path = '../data/binding/splits/'
oversample = SMOTE()
trainX_normSMOTE_temp, trainy_normSMOTE_temp = oversample.fit_resample(train_x, train_y)
trainX_normSMOTE = trainX_normSMOTE_temp
trainy_normSMOTE = trainy_normSMOTE_temp.reshape(-1)

train_x_dfSMOTE = pd.DataFrame(trainX_normSMOTE) 
train_y_dfSMOTE = pd.DataFrame(trainy_normSMOTE.reshape(-1)) 
test_y_df = pd.DataFrame(test_y)
test_x_df = pd.DataFrame(test_x)
train_x_dfSMOTE.shape, train_y_dfSMOTE.shape, test_y_df.shape, test_x_df.shape

train_x_dfSMOTE.to_csv(split_path+'bind_train_x_SMOTE.csv', index=False, header=False)
train_y_dfSMOTE.to_csv(split_path+'bind_train_y_SMOTE.csv', index=False, header=False) 
test_y_df.to_csv(split_path+'bind_test_y_SMOTE.csv', index=False, header=False) 
test_x_df.to_csv(split_path+'bind_test_x_SMOTE.csv',index=False, header=False) 

# RandomUnderSampler 

In [33]:

undersample = RandomUnderSampler()
train_x_UNDER, train_y_UNDER = undersample.fit_resample(train_x, train_y)

train_y_UNDER = train_y_UNDER.flatten()

train_x_dfUNDER = pd.DataFrame(train_x_UNDER) 
train_y_dfUNDER = pd.DataFrame(train_y_UNDER) 
test_y_df = pd.DataFrame(test_y)
test_x_df = pd.DataFrame(test_x)

train_x_dfUNDER.to_csv(split_path+'bind_train_x_UNDER.csv', index=False, header=False)
train_y_dfUNDER.to_csv(split_path+'bind_train_y_UNDER.csv', index=False, header=False) 
test_y_df.to_csv(split_path+'bind_test_y_UNDER.csv', index=False, header=False) 
test_x_df.to_csv(split_path+'bind_test_x_UNDER.csv', index=False, header=False) 

In [34]:
adasyn = ADASYN() 
train_x_tempADASYN, train_y_tempADASYN = adasyn.fit_resample(train_x, train_y)
train_xADASYN = train_x_tempADASYN
train_yADASYN = train_y_tempADASYN.flatten()

train_xADASYN.shape,train_yADASYN.shape, testX_norm.shape, test_y.shape

train_x_dfADASYN = pd.DataFrame(train_xADASYN) 
train_y_dfADASYN = pd.DataFrame(train_yADASYN.reshape(-1)) 
test_y_df = pd.DataFrame(test_y.reshape(-1))
test_x_df = pd.DataFrame(testX_norm)

train_x_dfADASYN.to_csv(split_path+'bind_train_x_ADASYN.csv', index=False, header=False)
train_y_dfADASYN.to_csv(split_path+'bind_train_y_ADASYN.csv', index=False, header=False) 
test_y_df.to_csv(split_path+'bind_test_y_ADASYN.csv', index=False, header=False) 
test_x_df.to_csv(split_path+'bind_test_x_ADASYN.csv', index=False, header=False) 

# Repeat for Morgan FP 
change radius if you want to test r=2 or r=3 

In [35]:
X_morgan_train 
X_morgan_test = X_morgan_test
y_trainmorgan = smile_prop_train['active']
y_testmorgan = smile_prop_test['active']

train_x_dfmorgan = pd.DataFrame(X_morgan_train) 
train_y_dfmorgan = pd.DataFrame(y_trainmorgan) 
test_y_dfmorgan = pd.DataFrame(y_testmorgan)
test_x_dfmorgan = pd.DataFrame(X_morgan_test)

train_x_dfmorgan.to_csv(split_path+'bind_train_x_Morgan2.csv', index=False, header=False)
train_y_dfmorgan.to_csv(split_path+'bind_train_y_Morgan2.csv', index=False, header=False) 
test_y_dfmorgan.to_csv(split_path+'bind_test_y_Morgan2.csv', index=False, header=False) 
test_x_dfmorgan.to_csv(split_path+'bind_test_x_Morgan2.csv', index=False, header=False) 
print(train_x_dfmorgan.shape, train_y_dfmorgan.shape, test_y_dfmorgan.shape, test_x_dfmorgan.shape)

(1125, 2048) (1125, 1) (283, 1) (283, 2048)


# SMOTE on Morgan fp

In [36]:
oversample = SMOTE()
trainX_Morgan_SMOTE, trainy_Morgan_SMOTE = oversample.fit_resample(X_morgan_train,train_y)
train_x_dfmorganSMOTE = pd.DataFrame(trainX_Morgan_SMOTE) 
train_y_dfmorganSMOTE = pd.DataFrame(trainy_Morgan_SMOTE) 
test_y_dfmorganSMOTE = pd.DataFrame(y_testmorgan)
test_x_dfmorganSMOTE = pd.DataFrame(X_morgan_test)

train_x_dfmorganSMOTE.to_csv(split_path+'bind_train_x_Morgan2SMOTE.csv', index=False, header=False)
train_y_dfmorganSMOTE.to_csv(split_path+'bind_train_y_Morgan2SMOTE.csv', index=False, header=False) 
test_y_dfmorganSMOTE.to_csv(split_path+'bind_test_y_Morgan2SMOTE.csv', index=False, header=False) 
test_x_dfmorganSMOTE.to_csv(split_path+'bind_test_x_Morgan2SMOTE.csv', index=False, header=False) 
print(train_x_dfmorganSMOTE.shape, train_y_dfmorganSMOTE.shape, test_x_dfmorganSMOTE.shape, test_x_dfmorganSMOTE.shape)

(2160, 2048) (2160, 1) (283, 2048) (283, 2048)


# Undersample 

In [37]:
undersample = RandomUnderSampler()
train_x_Morgan_UNDER, train_y_Morgan_UNDER = undersample.fit_resample(X_morgan_train, train_y)

train_x_dfMorganUNDER = pd.DataFrame(train_x_Morgan_UNDER) 
train_y_dfMorganUNDER = pd.DataFrame(train_y_Morgan_UNDER) 
test_y_dfMorganUnder = pd.DataFrame(y_testmorgan)
test_x_dfMorganUnder = pd.DataFrame(X_morgan_test)

train_x_dfMorganUNDER.to_csv(split_path+'bind_train_x_Morgan2UNDER.csv', index=False, header=False)
train_y_dfMorganUNDER.to_csv(split_path+'bind_train_y_Morgan2UNDER.csv', index=False, header=False) 
test_y_dfMorganUnder.to_csv(split_path+'bind_test_y_Morgan2UNDER.csv', index=False, header=False) 
test_x_dfMorganUnder.to_csv(split_path+'bind_test_x_Morgan2UNDER.csv', index=False, header=False)
print(train_x_dfMorganUNDER.shape, train_y_dfMorganUNDER.shape, test_x_dfMorganUnder.shape, test_y_dfMorganUnder.shape)

(90, 2048) (90, 1) (283, 2048) (283, 1)


# ADASYN 

In [38]:
adasyn = ADASYN() 
train_x_MorganADASYN, train_y_MorganADASYN = adasyn.fit_resample(X_morgan_train, train_y)

train_x_dfMorganADASYN = pd.DataFrame(train_x_MorganADASYN) 
train_y_dfMorganADASYN = pd.DataFrame(train_y_MorganADASYN) 
test_y_dfMorganADASYN = pd.DataFrame(y_testmorgan)
test_x_dfMorganADASYN = pd.DataFrame(X_morgan_test)

train_x_dfMorganADASYN.to_csv(split_path+'bind_train_x_Morgan2ADASYN.csv', index=False, header=False)
train_y_dfMorganADASYN.to_csv(split_path+'bind_train_y_Morgan2ADASYN.csv', index=False, header=False) 
test_y_dfMorganADASYN.to_csv(split_path+'bind_test_y_Morgan2ADASYN.csv', index=False, header=False) 
test_x_dfMorganADASYN.to_csv(split_path+'bind_test_x_Morgan2ADASYN.csv', index=False, header=False) 

print(train_x_dfMorganADASYN.shape, train_y_dfMorganADASYN.shape, test_x_dfMorganADASYN.shape, test_y_dfMorganADASYN.shape)

(2165, 2048) (2165, 1) (283, 2048) (283, 1)


# Rinse and repeat for inhibition